In [1]:
import pandas as pd 
import numpy as np 
# import matplotlib.pyplot as plt
import seaborn as sns
from procan_connectome.config import DATA_PATH, PLOT_PATH, RANDOM_STATE, LOGGER_LEVEL
from procan_connectome.data_ingestion.ingest_data import get_dataset
import os
import re
import logging
# import umap.umap_ as umap
from sklearn.cluster import DBSCAN
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import xgboost as xgb
from sklearn.decomposition import PCA 
from procan_connectome.data_processing.correlation_filter import CorrelationFilter
from procan_connectome.data_processing.linear_svc_importance_filter import LinearSVCImportanceFilter
from procan_connectome.data_processing.select_k_best_filter import SelectKBestFilter
from procan_connectome.data_processing.outlier_clipper import OutlierClipper
from procan_connectome.data_processing.powertransformer_wrapper import PowerTransformerWrapper
from procan_connectome.data_processing.rf_importance_filter import RFImportanceFilter
from procan_connectome.data_processing.umap_transformer import UmapTransformer
from sklearn.pipeline import Pipeline


In [2]:
CORR_THRESHOLD=0.99
DATASET_FILE_NAME = 'combined_datasets.csv'
df = get_dataset(DATASET_FILE_NAME)
logging.basicConfig(level=LOGGER_LEVEL)
df

,fun-HeadMovement,label,fun-SumEdgeNum_Between_Module02_05,fun-SumEdgeNum_Between_Module04_07,fun-SumEdgeNum_Between_Module01_08,fun-SumEdgeNum_Between_Module04_05,fun-SumEdgeNum_Within_Module03,fun-SumEdgeNum_Within_Module02,fun-SumEdgeNum_Between_Module03_05,fun-SumEdgeNum_Between_Module02_06,...,str-CustomPc_normalized-Temporal Sup L,str-CustomPc_normalized-Temporal Sup R,str-CustomPc_normalized-Temporal Pole Sup L,str-CustomPc_normalized-Temporal Pole Sup R,str-CustomPc_normalized-Temporal Mid L,str-CustomPc_normalized-Temporal Mid R,str-CustomPc_normalized-Temporal Pole Mid L,str-CustomPc_normalized-Temporal Pole Mid R,str-CustomPc_normalized-Temporal Inf L,str-CustomPc_normalized-Temporal Inf R
ID,,,,,,,,,,,,,,,,,,,,,
CAL_004,0.045966,Stage_0,48.0,12.0,21.0,14.0,4.0,75.0,34.0,55.0,...,0.982290,0.963543,0.971817,0.964923,0.983479,0.941668,0.962406,0.940476,0.947735,0.959402
CAL_005,0.047396,Stage_1b,57.0,27.0,0.0,30.0,7.0,81.0,17.0,9.0,...,0.973570,0.959821,0.980544,0.963294,0.992945,0.959436,0.972796,0.941429,0.975446,0.916014
CAL_007,0.078991,Stage_1b,53.0,37.0,17.0,24.0,3.0,59.0,21.0,21.0,...,0.976676,0.895044,0.976801,0.970409,0.981110,0.969540,0.965412,0.943672,0.972185,0.957062
CAL_008,0.077669,Stage_1a,37.0,13.0,27.0,24.0,4.0,76.0,26.0,26.0,...,0.989446,0.996458,0.976290,0.968382,0.965572,0.980069,0.919323,0.959067,0.948494,0.973390
CAL_009,0.082430,Stage_1b,87.0,35.0,13.0,38.0,3.0,63.0,15.0,22.0,...,0.970273,0.980159,0.966531,0.959067,0.974329,0.975714,0.929165,0.921749,0.959631,0.929818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SB_040,0.096126,Stage_1a,52.0,18.0,19.0,28.0,6.0,73.0,8.0,59.0,...,0.953966,0.950581,0.973880,0.967878,0.975425,0.978427,0.968466,0.940408,0.971429,0.950915
SB_041,0.087158,Stage_1b,41.0,29.0,12.0,13.0,7.0,85.0,21.0,52.0,...,0.982290,0.949841,0.979753,0.962463,0.980298,0.948436,0.956694,0.931649,0.947108,0.930342
SB_042,0.064763,Stage_0,93.0,36.0,10.0,46.0,5.0,74.0,22.0,20.0,...,0.978427,0.939886,0.958648,0.979818,0.971193,0.971721,0.895924,0.946609,0.938571,0.951063


In [3]:
X, y = df.drop(columns='label'), df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [4]:
"""What do we know about this dataset? 
* High dimensionaliy
* Low sample count
* High correlation between features

Proposed SVC Pipe: 
* Drop correlated 
* Power Transform 
* Feature selection 
* 
"""

'What do we know about this dataset? \n* High dimensionaliy\n* Low sample count\n* High correlation between features\n\nProposed SVC Pipe: \n* Drop \n'

In [18]:
from itertools import product
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.decomposition import *
from sklearn.base import clone


def get_type_name(obj):
    name = type(obj).__name__
    return name 

def steps_builder(transforms:list)->list: 
    steps = [(get_type_name(x), x) for x in transforms]
    # steps_clean_names = [x+str(i) for (x,i) in list(zip(steps, range(len(steps))))]
    return steps 

def clean_duplicate_step_names(steps)-> list: 
    inventory = {}
    revised_steps =[]
    for step in steps: 
        if step in inventory: 
            inventory[step]+=1
            step = (step[0]+str(inventory[step]), step[1])
        else: 
            inventory[step] = 0
        revised_steps.append(step)
    return revised_steps 

def get_pipes(scalers, filters, reducers): 
    pipes = [Pipeline(clean_duplicate_step_names(x)) for x in product(
                                                                    steps_builder(scalers),
                                                                    steps_builder(filters),
                                                                    steps_builder(reducers))]
    return pipes 


alpha = 0.1
threshold = 0.001
n_components =80
k=300
estimator = RandomForestClassifier(random_state=RANDOM_STATE)
scalers = [None,
           StandardScaler(),
           MinMaxScaler(), 
           MaxAbsScaler(),
           Normalizer(),
           PowerTransformer(),
           RobustScaler()
           ]

filters = [None,
           SelectKBest(k=k),
           SelectFpr(alpha=alpha),
        #    RFE(clone(estimator), step=10),
        #    RFECV(clone(estimator), step=10),
           SelectFromModel(clone(estimator), threshold=threshold)
           ]

reducers = [None,
            PCA(n_components=n_components, svd_solver='full', random_state=RANDOM_STATE),
            FastICA(n_components=n_components, random_state = RANDOM_STATE),
            IncrementalPCA(n_components=n_components),
            KernelPCA(n_components=n_components, random_state=RANDOM_STATE),
            # NMF(n_components=n_components, random_state=RANDOM_STATE)
           ]


pipes = get_pipes(scalers, filters, reducers)
rf = RandomForestClassifier(random_state = RANDOM_STATE)
evaluator = PipelineEvaluator(pipes, rf, test_size = 0.5)
evaluator.fit(X, y)
results = evaluator.get_results()
results


INFO:root:Evaluating No_pipeline pipeline...
INFO:root:Evaluating NoneType-NoneType1-NoneType2 pipeline...
INFO:root:Evaluating NoneType-NoneType1-PCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-FastICA pipeline...
INFO:root:Evaluating NoneType-NoneType1-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-KernelPCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-NoneType1 pipeline...
INFO:root:Evaluating NoneType-SelectKBest-PCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-FastICA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-KernelPCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-NoneType1 pipeline...
INFO:root:Evaluating NoneType-SelectFpr-PCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-FastICA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-KernelPCA pipeline...
INFO:root:Evaluating Non

,Pipeline_name,Score
0,No_pipeline,0.337209
1,NoneType-NoneType1-NoneType2,0.337209
2,NoneType-NoneType1-PCA,0.372093
3,NoneType-NoneType1-FastICA,0.372093
4,NoneType-NoneType1-IncrementalPCA,0.348837
...,...,...
136,RobustScaler-SelectFromModel-NoneType,0.372093
137,RobustScaler-SelectFromModel-PCA,0.337209
138,RobustScaler-SelectFromModel-FastICA,0.325581
139,RobustScaler-SelectFromModel-IncrementalPCA,0.395349


In [22]:
results.sort_values(by='Score', ascending=False)

,Pipeline_name,Score
106,PowerTransformer-SelectKBest-NoneType,0.418605
68,MaxAbsScaler-SelectKBest-FastICA,0.406977
139,RobustScaler-SelectFromModel-IncrementalPCA,0.395349
57,MinMaxScaler-SelectFromModel-PCA,0.383721
65,MaxAbsScaler-NoneType-KernelPCA,0.383721
...,...,...
88,Normalizer-SelectKBest-FastICA,0.209302
13,NoneType-SelectFpr-FastICA,0.197674
93,Normalizer-SelectFpr-FastICA,0.197674
113,PowerTransformer-SelectFpr-FastICA,0.174419


In [17]:
from itertools import product
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.decomposition import *
from sklearn.base import clone
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)

def get_type_name(obj):
    name = type(obj).__name__
    return name 

def steps_builder(transforms:list)->list: 
    steps = [(get_type_name(x), x) for x in transforms]
    # steps_clean_names = [x+str(i) for (x,i) in list(zip(steps, range(len(steps))))]
    return steps 

def clean_duplicate_step_names(steps)-> list: 
    inventory = {}
    revised_steps =[]
    for step in steps: 
        if step in inventory: 
            inventory[step]+=1
            step = (step[0]+str(inventory[step]), step[1])
        else: 
            inventory[step] = 0
        revised_steps.append(step)
    return revised_steps 

def get_pipes(scalers, filters, reducers): 
    pipes = [Pipeline(clean_duplicate_step_names(x)) for x in product(
                                                                    steps_builder(scalers),
                                                                    steps_builder(filters),
                                                                    steps_builder(reducers))]
    return pipes 


alpha = 0.1
threshold = 0.001
n_components =80
k=100
# estimator = RandomForestClassifier(random_state=RANDOM_STATE)
estimator = XGBClassifier(objective="multi:softprob",
                          n_jobs=8,
                          random_state=RANDOM_STATE,
                          eval_metric="auc",
                          use_label_encoder=False)  # Good for imbalanced classes

scalers = [None,
           StandardScaler(),
           MinMaxScaler(), 
           MaxAbsScaler(),
           Normalizer(),
           PowerTransformer(),
           RobustScaler()
           ]

filters = [None,
           SelectKBest(k=k),
           SelectFpr(alpha=alpha),
        #    RFE(clone(estimator), step=10),
        #    RFECV(clone(estimator), step=10),
           SelectFromModel(clone(estimator), threshold=threshold)
           ]

reducers = [None,
            PCA(n_components=n_components, svd_solver='full', random_state=RANDOM_STATE),
            FastICA(n_components=n_components, random_state = RANDOM_STATE),
            IncrementalPCA(n_components=n_components),
            KernelPCA(n_components=n_components, random_state=RANDOM_STATE),
            # NMF(n_components=n_components, random_state=RANDOM_STATE)
           ]


pipes = get_pipes(scalers, filters, reducers)
# rf = RandomForestClassifier(random_state = RANDOM_STATE)
evaluator = PipelineEvaluator(pipes, estimator, test_size = 0.5)
evaluator.fit(X, le.fit_transform(y))
results = evaluator.get_results()
results.sort_values(by='Score', ascending=False)


INFO:root:Evaluating No_pipeline pipeline...
INFO:root:Evaluating NoneType-NoneType1-NoneType2 pipeline...
INFO:root:Evaluating NoneType-NoneType1-PCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-FastICA pipeline...
INFO:root:Evaluating NoneType-NoneType1-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-KernelPCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-NoneType1 pipeline...
INFO:root:Evaluating NoneType-SelectKBest-PCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-FastICA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-KernelPCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-NoneType1 pipeline...
INFO:root:Evaluating NoneType-SelectFpr-PCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-FastICA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-KernelPCA pipeline...
INFO:root:Evaluating Non

,Pipeline_name,Score
63,MaxAbsScaler-NoneType-FastICA,0.360465
106,PowerTransformer-SelectKBest-NoneType,0.360465
104,PowerTransformer-NoneType-IncrementalPCA,0.348837
102,PowerTransformer-NoneType-PCA,0.348837
105,PowerTransformer-NoneType-KernelPCA,0.348837
...,...,...
118,PowerTransformer-SelectFromModel-FastICA,0.174419
38,StandardScaler-SelectFromModel-FastICA,0.174419
33,StandardScaler-SelectFpr-FastICA,0.162791
68,MaxAbsScaler-SelectKBest-FastICA,0.162791


In [23]:
from itertools import product
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.decomposition import *
from sklearn.base import clone
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)

def get_type_name(obj):
    name = type(obj).__name__
    return name 

def steps_builder(transforms:list)->list: 
    steps = [(get_type_name(x), x) for x in transforms]
    # steps_clean_names = [x+str(i) for (x,i) in list(zip(steps, range(len(steps))))]
    return steps 

def clean_duplicate_step_names(steps)-> list: 
    inventory = {}
    revised_steps =[]
    for step in steps: 
        if step in inventory: 
            inventory[step]+=1
            step = (step[0]+str(inventory[step]), step[1])
        else: 
            inventory[step] = 0
        revised_steps.append(step)
    return revised_steps 

def get_pipes(scalers, filters, reducers): 
    pipes = [Pipeline(clean_duplicate_step_names(x)) for x in product(
                                                                    steps_builder(scalers),
                                                                    steps_builder(filters),
                                                                    steps_builder(reducers))]
    return pipes 


alpha = 0.1
threshold = 0.001
n_components =80
k=100
# estimator = RandomForestClassifier(random_state=RANDOM_STATE)
estimator = XGBClassifier(objective="multi:softprob",
                          n_jobs=8,
                          random_state=RANDOM_STATE,
                          eval_metric="auc",
                          use_label_encoder=False)  # Good for imbalanced classes

scalers = [None,
           StandardScaler(),
           MinMaxScaler(), 
           MaxAbsScaler(),
           Normalizer(),
           PowerTransformer(),
           RobustScaler()
           ]

filters = [None,
           SelectKBest(k=k),
           SelectFpr(alpha=alpha),
        #    RFE(clone(estimator), step=10),
        #    RFECV(clone(estimator), step=10),
           SelectFromModel(clone(estimator), threshold=threshold)
           ]

reducers = [None,
            PCA(n_components=n_components, svd_solver='full', random_state=RANDOM_STATE),
            FastICA(n_components=n_components, random_state = RANDOM_STATE),
            IncrementalPCA(n_components=n_components),
            KernelPCA(n_components=n_components, random_state=RANDOM_STATE),
            # NMF(n_components=n_components, random_state=RANDOM_STATE)
           ]


pipes = get_pipes(scalers, filters, reducers)
# rf = RandomForestClassifier(random_state = RANDOM_STATE)
evaluator = PipelineEvaluator(pipes, estimator, test_size = 0.5)
evaluator.fit(X, le.fit_transform(y))
results = evaluator.get_results()
results.sort_values(by='Score', ascending=False)


ID
CAL_004     Stage_0
CAL_005    Stage_1b
CAL_007    Stage_1b
CAL_008    Stage_1a
CAL_009    Stage_1b
             ...   
SB_040     Stage_1a
SB_041     Stage_1b
SB_042      Stage_0
SB_043      Stage_0
SB_044      Stage_0
Name: label, Length: 171, dtype: object

In [9]:
from itertools import product
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import *
from sklearn.base import clone


def get_type_name(obj):
    name = type(obj).__name__
    return name 

def steps_builder(transforms:list)->list: 
    steps = [(get_type_name(x), x) for x in transforms]
    # steps_clean_names = [x+str(i) for (x,i) in list(zip(steps, range(len(steps))))]
    return steps 

def clean_duplicate_step_names(steps)-> list: 
    inventory = {}
    revised_steps =[]
    for step in steps: 
        if step in inventory: 
            inventory[step]+=1
            step = (step[0]+str(inventory[step]), step[1])
        else: 
            inventory[step] = 0
        revised_steps.append(step)
    return revised_steps 

def get_pipes(scalers, filters, reducers): 
    pipes = [Pipeline(clean_duplicate_step_names(x)) for x in product(
                                                                    steps_builder(scalers),
                                                                    steps_builder(filters),
                                                                    steps_builder(reducers))]
    return pipes 


alpha = 0.1
threshold = 0.001
n_components =80
k=300
estimator = LogisticRegression(random_state=RANDOM_STATE, class_weight="balanced", solver="newton-cg")

scalers = [None,
           StandardScaler(),
           MinMaxScaler(), 
           MaxAbsScaler(),
           Normalizer(),
           PowerTransformer(),
           RobustScaler()
           ]

filters = [None,
           SelectKBest(k=k),
           SelectFpr(alpha=alpha),
        #    RFE(clone(estimator), step=10),
        #    RFECV(clone(estimator), step=10),
           SelectFromModel(clone(estimator), threshold=threshold)
           ]

reducers = [None,
            PCA(n_components=n_components, svd_solver='full', random_state=RANDOM_STATE),
            FastICA(n_components=n_components, random_state = RANDOM_STATE),
            IncrementalPCA(n_components=n_components),
            KernelPCA(n_components=n_components, random_state=RANDOM_STATE),
            # NMF(n_components=n_components, random_state=RANDOM_STATE)
           ]


pipes = get_pipes(scalers, filters, reducers)
# rf = RandomForestClassifier(random_state = RANDOM_STATE)
evaluator = PipelineEvaluator(pipes, estimator, test_size = 0.5)
evaluator.fit(X, y)
results = evaluator.get_results()
results.sort_values(by='Score', ascending=False)


INFO:root:Evaluating No_pipeline pipeline...
INFO:root:Evaluating NoneType-NoneType1-NoneType2 pipeline...
INFO:root:Evaluating NoneType-NoneType1-PCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-FastICA pipeline...
INFO:root:Evaluating NoneType-NoneType1-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-KernelPCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-NoneType1 pipeline...
INFO:root:Evaluating NoneType-SelectKBest-PCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-FastICA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-KernelPCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-NoneType1 pipeline...
INFO:root:Evaluating NoneType-SelectFpr-PCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-FastICA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-KernelPCA pipeline...
INFO:root:Evaluating Non

,Pipeline_name,Score
23,StandardScaler-NoneType-FastICA,0.395349
38,StandardScaler-SelectFromModel-FastICA,0.395349
103,PowerTransformer-NoneType-FastICA,0.395349
118,PowerTransformer-SelectFromModel-FastICA,0.395349
116,PowerTransformer-SelectFromModel-NoneType,0.383721
...,...,...
88,Normalizer-SelectKBest-FastICA,0.197674
53,MinMaxScaler-SelectFpr-FastICA,0.174419
33,StandardScaler-SelectFpr-FastICA,0.162791
73,MaxAbsScaler-SelectFpr-FastICA,0.151163


In [10]:
from sklearn.naive_bayes import GaussianNB
alpha = 0.1
threshold = 0.001
n_components =80
k=300
estimator = GaussianNB()

scalers = [None,
           StandardScaler(),
           MinMaxScaler(), 
           MaxAbsScaler(),
           Normalizer(),
           PowerTransformer(),
           RobustScaler()
           ]

filters = [None,
           SelectKBest(k=k),
           SelectFpr(alpha=alpha),
        #    RFE(clone(estimator), step=10),
        #    RFECV(clone(estimator), step=10),
        #    SelectFromModel(clone(estimator), threshold=threshold)
           ]

reducers = [None,
            PCA(n_components=n_components, svd_solver='full', random_state=RANDOM_STATE),
            FastICA(n_components=n_components, random_state = RANDOM_STATE),
            IncrementalPCA(n_components=n_components),
            KernelPCA(n_components=n_components, random_state=RANDOM_STATE),
            # NMF(n_components=n_components, random_state=RANDOM_STATE)
           ]


pipes = get_pipes(scalers, filters, reducers)
# rf = RandomForestClassifier(random_state = RANDOM_STATE)
evaluator = PipelineEvaluator(pipes, estimator, test_size = 0.5)
evaluator.fit(X, y)
results = evaluator.get_results()
results.sort_values(by='Score', ascending=False)


INFO:root:Evaluating No_pipeline pipeline...
INFO:root:Evaluating NoneType-NoneType1-NoneType2 pipeline...
INFO:root:Evaluating NoneType-NoneType1-PCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-FastICA pipeline...
INFO:root:Evaluating NoneType-NoneType1-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-KernelPCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-NoneType1 pipeline...
INFO:root:Evaluating NoneType-SelectKBest-PCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-FastICA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-KernelPCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-NoneType1 pipeline...
INFO:root:Evaluating NoneType-SelectFpr-PCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-FastICA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-SelectFpr-KernelPCA pipeline...
INFO:root:Evaluating Sta

,Pipeline_name,Score
20,StandardScaler-NoneType-KernelPCA,0.395349
19,StandardScaler-NoneType-IncrementalPCA,0.395349
17,StandardScaler-NoneType-PCA,0.395349
85,PowerTransformer-SelectKBest-KernelPCA,0.383721
84,PowerTransformer-SelectKBest-IncrementalPCA,0.383721
...,...,...
88,PowerTransformer-SelectFpr-FastICA,0.151163
43,MinMaxScaler-SelectFpr-FastICA,0.069767
58,MaxAbsScaler-SelectFpr-FastICA,0.069767
13,NoneType-SelectFpr-FastICA,0.046512


In [18]:
from itertools import product
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.decomposition import *
from sklearn.base import clone


def get_type_name(obj):
    name = type(obj).__name__
    return name 

def steps_builder(transforms:list)->list: 
    steps = [(get_type_name(x), x) for x in transforms]
    # steps_clean_names = [x+str(i) for (x,i) in list(zip(steps, range(len(steps))))]
    return steps 

def clean_duplicate_step_names(steps)-> list: 
    inventory = {}
    revised_steps =[]
    for step in steps: 
        if step in inventory: 
            inventory[step]+=1
            step = (step[0]+str(inventory[step]), step[1])
        else: 
            inventory[step] = 0
        revised_steps.append(step)
    return revised_steps 

def get_pipes(scalers, filters, reducers): 
    pipes = [Pipeline(clean_duplicate_step_names(x)) for x in product(
                                                                    steps_builder(scalers),
                                                                    steps_builder(filters),
                                                                    steps_builder(reducers))]
    return pipes 


alpha = 0.1
threshold = 0.001
n_components =80
k=300
estimator = LinearSVC(random_state=RANDOM_STATE)
scalers = [None,
           StandardScaler(),
           MinMaxScaler(), 
           MaxAbsScaler(),
           Normalizer(),
           PowerTransformer(),
           RobustScaler()
           ]

filters = [None,
           SelectKBest(k=k),
           SelectFpr(alpha=alpha),
        #    RFE(clone(estimator), step=10),
        #    RFECV(clone(estimator), step=10),
           SelectFromModel(clone(estimator), threshold=threshold)
           ]

reducers = [None,
            PCA(n_components=n_components, svd_solver='full', random_state=RANDOM_STATE),
            FastICA(n_components=n_components, random_state = RANDOM_STATE),
            IncrementalPCA(n_components=n_components),
            KernelPCA(n_components=n_components, random_state=RANDOM_STATE),
            # NMF(n_components=n_components, random_state=RANDOM_STATE)
           ]


pipes = get_pipes(scalers, filters, reducers)
svc = LinearSVC(random_state=RANDOM_STATE)
evaluator = PipelineEvaluator(pipes, svc, test_size = 0.5)
evaluator.fit(X, y)
results = evaluator.get_results()
results


INFO:root:Evaluating No_pipeline pipeline...
INFO:root:Evaluating NoneType-NoneType1-NoneType2 pipeline...
INFO:root:Evaluating NoneType-NoneType1-PCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-FastICA pipeline...
INFO:root:Evaluating NoneType-NoneType1-IncrementalPCA pipeline...
INFO:root:Evaluating NoneType-NoneType1-KernelPCA pipeline...
INFO:root:Evaluating NoneType-SelectKBest-NoneType1 pipeline...
C:\Users\Mike\anaconda3\envs\ML\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
INFO:root:Evaluating NoneType-SelectKBest-PCA pipeline...
C:\Users\Mike\anaconda3\envs\ML\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
INFO:root:Evaluating NoneType-SelectKBest-FastICA pipeline...
INFO:root:Evaluating NoneType-SelectKBes

KeyboardInterrupt: 

In [11]:
results.sort_values(by='Score', ascending=False)

,Pipeline_name,Score
58,MinMaxScaler-SelectFromModel-FastICA,0.418605
43,MinMaxScaler-NoneType-FastICA,0.418605
63,MaxAbsScaler-NoneType-FastICA,0.395349
42,MinMaxScaler-NoneType-PCA,0.395349
44,MinMaxScaler-NoneType-IncrementalPCA,0.395349
...,...,...
4,NoneType-NoneType1-IncrementalPCA,0.232558
2,NoneType-NoneType1-PCA,0.232558
88,Normalizer-SelectKBest-FastICA,0.232558
13,NoneType-SelectFpr-FastICA,0.220930


In [10]:
from itertools import product
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.decomposition import *
from sklearn.base import clone

kbest = SelectKBest(k=300)
kbest.fit(X, y)
supp = kbest.get_support()
X.columns[supp]

Index(['fun-SumEdgeNum_Within_Module03', 'fun-SumEdgeNum_Within_Module06',
       'fun-SumEdgeNum_Within_Module07', 'fun-SumEdgeNum_Within_Module05',
       'fun-SumEdgeNum_Between_Module02_03', 'fun-Bc-Precentral R',
       'fun-Bc-Frontal Sup R', 'fun-Bc-Frontal Sup Orb R',
       'fun-Bc-Frontal Mid L', 'fun-Bc-Rolandic Oper R',
       ...
       'str-CustomPc_normalized-Occipital Sup L',
       'str-CustomPc_normalized-Occipital Sup R',
       'str-CustomPc_normalized-Fusiform L',
       'str-CustomPc_normalized-Parietal Sup L',
       'str-CustomPc_normalized-Parietal Inf R',
       'str-CustomPc_normalized-Angular L',
       'str-CustomPc_normalized-Paracentral Lobule L',
       'str-CustomPc_normalized-Paracentral Lobule R',
       'str-CustomPc_normalized-Putamen L',
       'str-CustomPc_normalized-Temporal Pole Mid R'],
      dtype='object', length=300)

In [11]:
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, SelectKBest

pipes = [
    Pipeline([("cf", CorrelationFilter(threshold=0.9)), ('pt', PowerTransformerWrapper())]),
    Pipeline([('ss', StandardScaler()),]),
    # Pipeline([("RFE", RFE(RandomForestClassifier(), n_features_to_select=300, step=1)),]),
    Pipeline([("PCA", PCA(n_components=0.95, svd_solver='full', random_state=RANDOM_STATE)),]),
    Pipeline([
        ('cf', CorrelationFilter(threshold=0.9)),
        # ('pt', PowerTransformerWrapper()),
        ('rf_filter', RFImportanceFilter(threshold=0.001)),

    ])
]
rf = RandomForestClassifier(random_state = RANDOM_STATE)
evaluator = PipelineEvaluator(pipes, rf, test_size = 0.2)
evaluator.fit(X_train, y_train)
results = evaluator.get_results()
results

INFO:root:Evaluating No_pipeline pipeline...
INFO:root:Evaluating cf-pt pipeline...
INFO:root:Found 546 correlated features to drop
/home/mklasby/miniconda3/envs/ensf-ml/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:3237: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
INFO:root:PowerTransformer fit to 813 features.
INFO:root:Evaluating ss pipeline...
INFO:root:Evaluating PCA pipeline...
INFO:root:Evaluating cf-rf_filter pipeline...
INFO:root:Found 546 correlated features to drop
INFO:root:Found 433 important features.
INFO:root:Evaluation complete on 4 pipelines.


,Pipeline_name,Score
0,No_pipeline,0.464286
1,cf-pt,0.428571
2,ss,0.428571
3,PCA,0.357143
4,cf-rf_filter,0.500000


In [65]:
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, SelectKBest, SelectFdr, SelectFpr, SelectFwe

alpha=0.1

pipes = [
    Pipeline([
        ('cf', CorrelationFilter(threshold=0.9)),
        ('pt', PowerTransformerWrapper()),
        ('rf_filter', RFImportanceFilter(threshold=0.001, random_state=RANDOM_STATE)),
        ('pca', PCA(n_components=0.95, svd_solver='full', random_state=RANDOM_STATE))
    ]),
    Pipeline([
        ('cf', CorrelationFilter(threshold=0.9)),
        ('pt', PowerTransformerWrapper()),
        ('kbest', SelectKBest(k=300)),
        ('pca', PCA(n_components=0.95, svd_solver='full', random_state=RANDOM_STATE))
    ]),
    # Pipeline([
    #     ('SelectFdr', SelectFdr(alpha=alpha))
    # ]),
    Pipeline([
        ('SelectFpr', SelectFpr(alpha=alpha))
    ]),
    # Pipeline([
    #     ('SelectFwe', SelectFwe(alpha=alpha))
    # ]),
]
rf = RandomForestClassifier(random_state = RANDOM_STATE)
evaluator = PipelineEvaluator(pipes, rf, test_size = 0.5)
evaluator.fit(X_train, y_train)
results = evaluator.get_results()
results

INFO:root:Evaluating No_pipeline pipeline...
INFO:root:Evaluating cf-pt-rf_filter-pca pipeline...
INFO:root:Found 548 correlated features to drop
INFO:root:PowerTransformer fit to 811 features.
INFO:root:Found 397 important features.
INFO:root:Evaluating cf-pt-kbest-pca pipeline...
INFO:root:Found 548 correlated features to drop
INFO:root:PowerTransformer fit to 811 features.
INFO:root:Evaluating SelectFpr pipeline...
INFO:root:Evaluation complete on 3 pipelines.


,Pipeline_name,Score
0,No_pipeline,0.323529
1,cf-pt-rf_filter-pca,0.382353
2,cf-pt-kbest-pca,0.338235
3,SelectFpr,0.352941


In [8]:
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, SelectKBest, SelectFdr, SelectFpr, SelectFwe
from sklearn.svm import LinearSVC

pipes = [
    Pipeline([
        ('cf', CorrelationFilter(threshold=0.9)),
        ('pt', PowerTransformerWrapper()),
        ('svc_filter', LinearSVCImportanceFilter(threshold=0.001)),

    ])
]
svc = LinearSVC(random_state=RANDOM_STATE, max_iter=100000)
evaluator = PipelineEvaluator(pipes, svc, test_size = 0.2, include_dummy_case=False)
evaluator.fit(X_train, y_train)
results = evaluator.get_results()
results

INFO:root:Evaluating No_pipeline pipeline...
INFO:root:Evaluating cf-pt pipeline...
INFO:root:Found 546 correlated features to drop
/home/mklasby/miniconda3/envs/ensf-ml/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:3237: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
INFO:root:PowerTransformer fit to 813 features.
INFO:root:Evaluating ss pipeline...
INFO:root:Evaluating PCA pipeline...
/home/mklasby/miniconda3/envs/ensf-ml/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
INFO:root:Evaluating cf-pt-svc_filter pipeline...
INFO:root:Found 546 correlated features to drop
/home/mklasby/miniconda3/envs/ensf-ml/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:3237: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
INFO:r

,Pipeline_name,Score
0,No_pipeline,0.392857
1,cf-pt,0.392857
2,ss,0.357143
3,PCA,0.214286
4,cf-pt-svc_filter,0.464286


In [10]:
# Global vars
from itertools import product
from procan_connectome.data_processing.pipeline_evaluator import PipelineEvaluator
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.decomposition import *
from sklearn.base import clone


def get_type_name(obj):
    name = type(obj).__name__
    return name 

def steps_builder(transforms:list)->list: 
    steps = [(get_type_name(x), x) for x in transforms]
    # steps_clean_names = [x+str(i) for (x,i) in list(zip(steps, range(len(steps))))]
    return steps 

def clean_duplicate_step_names(steps)-> list: 
    inventory = {}
    revised_steps =[]
    for step in steps: 
        if step in inventory: 
            inventory[step]+=1
            step = (step[0]+str(inventory[step]), step[1])
        else: 
            inventory[step] = 0
        revised_steps.append(step)
    return revised_steps 

def get_pipes(scalers, filters, reducers): 
    pipes = [Pipeline(clean_duplicate_step_names(x)) for x in product(
                                                                    steps_builder(scalers),
                                                                    steps_builder(filters),
                                                                    steps_builder(reducers))]
    return pipes 


alpha = 0.1
threshold = 0.001
n_components =15
k=20
estimator = RandomForestClassifier(random_state=RANDOM_STATE)
scalers = [None,
           StandardScaler(),
           MinMaxScaler(), 
           MaxAbsScaler(),
           Normalizer(),
           PowerTransformer(),
           RobustScaler()
           ]

filters = [None,
        #    SelectKBest(k=k),
        #    SelectFpr(alpha=alpha),
        #    RFE(clone(estimator), step=10),
        #    RFECV(clone(estimator), step=10),
        #    SelectFromModel(clone(estimator), threshold=threshold)
           ]

reducers = [None,
            # PCA(n_components=n_components, svd_solver='full', random_state=RANDOM_STATE),
            # FastICA(n_components=n_components, random_state = RANDOM_STATE),
            # IncrementalPCA(n_components=n_components),
            # KernelPCA(n_components=n_components, random_state=RANDOM_STATE),
            # # NMF(n_components=n_components, random_state=RANDOM_STATE)
           ]


pipes = get_pipes(scalers, filters, reducers)
rf = RandomForestClassifier(random_state = RANDOM_STATE)
evaluator = PipelineEvaluator(pipes, rf, test_size = 0.5)
evaluator.fit(X, y)
results = evaluator.get_results()
results


INFO:root:Evaluating No_pipeline pipeline...
INFO:root:Evaluating NoneType-NoneType1-NoneType2 pipeline...
INFO:root:Evaluating StandardScaler-NoneType-NoneType1 pipeline...
INFO:root:Evaluating MinMaxScaler-NoneType-NoneType1 pipeline...
INFO:root:Evaluating MaxAbsScaler-NoneType-NoneType1 pipeline...
INFO:root:Evaluating Normalizer-NoneType-NoneType1 pipeline...
INFO:root:Evaluating PowerTransformer-NoneType-NoneType1 pipeline...
INFO:root:Evaluating RobustScaler-NoneType-NoneType1 pipeline...
INFO:root:Evaluation complete on 7 pipelines.


,Pipeline_name,Score
0,No_pipeline,0.360465
1,NoneType-NoneType1-NoneType2,0.360465
2,StandardScaler-NoneType-NoneType1,0.360465
3,MinMaxScaler-NoneType-NoneType1,0.360465
4,MaxAbsScaler-NoneType-NoneType1,0.360465
5,Normalizer-NoneType-NoneType1,0.348837
6,PowerTransformer-NoneType-NoneType1,0.337209
7,RobustScaler-NoneType-NoneType1,0.360465
